In [6]:
## Imports ##
#------------------------------------------------#
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.autograd as autograd
#------------------------------------------------#
import matplotlib.pyplot as plt
import numpy as np
from time import time
import math
from scipy import ndimage, misc
import os
import sys
from tqdm import tqdm_notebook as tqdm
#------------------------------------------------#
from data import Dataset
from train_isola import SiameseNetwork

In [2]:
def rmd(img): # ReMove Depth channel
    return img[:,:3,:,:]

In [20]:
is_depth = True

if is_depth:
    model = torch.load('model_isola_graddepth').cuda()
else:    
    model = torch.load('model_isola_nodepth').cuda()

#Testing
batch_size = 256
device = 'cuda'
accuracy = []
trainset = Dataset(train=False, split_size=.8)
trainloader = DataLoader(dataset=trainset, batch_size=batch_size)

In [21]:
model.eval()
with torch.no_grad():
    for epoch in tqdm(range(5)):
        for i, (a, b, c) in tqdm(enumerate(trainloader)):
            ap = torch.FloatTensor(ndimage.sobel(a[:,:,:,-1])).unsqueeze(3).to(device).type(torch.float32)
            bp = torch.FloatTensor(ndimage.sobel(b[:,:,:,-1])).unsqueeze(3).to(device).type(torch.float32)
            a, b, c = (a.permute(0,3,1,2).to(device).type(torch.float32), 
                       b.permute(0,3,1,2).to(device).type(torch.float32), c.to(device).type(torch.float)) #for contrastive loss it is 0 for matching pairs	
            model.zero_grad()
            ap, bp = ap.permute(0,3,1,2), bp.permute(0,3,1,2)
            a = torch.cat([a[:,:3,:,:], ap], dim=1)
            b = torch.cat([b[:,:3,:,:], bp], dim=1)
            pred_c = model(a, b)
            pred_c = list(torch.round(pred_c).cpu().numpy())
            
            c = list(c.cpu().numpy())
            accuracy.append(sum([1 for x, y in zip(pred_c, c) if x == y]) / len(c))
    print('Accuracy:', sum(accuracy) / len(accuracy))

Accuracy: 0.7686553030303032


In [16]:
1-.378

0.622